In [1]:
import unittest
import matplotlib.pylab as plt
import sys
sys.path.append("C:/Users/lenovo/Desktop/spliking_nn_for_brain_simulation-master")
import unittest.mock as mock
import numpy as np
import os
import time
import torch
from multiprocessing.pool import Pool as pool
from multiprocessing.pool import ThreadPool as Thpool
import re
from numba import jit #加速器

In [2]:
def connect_for_single_sparse_block(block_idx, k, extern_input_rate, extern_input_k_sizes, degree=int(1e3), init_min=0, init_max=1, s = 0, e = -1):
    # extern_input_rate only works for E neurons.

    # this function is pertty slow.

    # we assume that the permution

    if e == -1:
        e = k
    assert 0 <= s < e <= k

    print("length:", e - s, "degree:", degree)
    print("generating weight")
    connect_weight = np.random.rand(e - s, degree, 2).astype(np.float32) * (init_max - init_min) + init_min
    print("generating idx")

    E_neuron_thresh = extern_input_k_sizes[block_idx]

    # connect_weight[is_E_neuron == 0] *= 4

    output_neuron_idx = np.broadcast_to(np.arange(s, e, dtype=np.uint32)[:, None], (e-s, degree))

    _extern_input_k_sizes = np.array(extern_input_k_sizes, dtype=np.int64)
    extern_input_rate = np.add.accumulate(extern_input_rate)[:-1]

    @jit(nogil=True, nopython=True)
    def _run(i):
        input_block_idx = np.zeros(degree, dtype=np.int16)#初始化
        input_channel_offset = np.zeros(degree, dtype=np.uint8)#初始化
        while True:
            k_idx = get_k_idx(k, degree, i)#不能与自己连接
            k_idx_is_E = (k_idx < E_neuron_thresh)##与E_neuron产生随机的连接关系？
            k_idx_is_I = (k_idx >= E_neuron_thresh)##与I_neuron产生随机的连接关系？
            if k_idx_is_I.shape[0] > 0:
                break
        # connect_weight[i, k_idx_is_I] *= connect_weight[i, k_idx_is_E].sum() / connect_weight[i, k_idx_is_I].sum()

        input_block_idx[k_idx_is_I] = block_idx#设置I_neuron

        r = np.random.rand(np.count_nonzero(k_idx_is_E))
        E_in_comming = np.searchsorted(extern_input_rate, r, 'right').astype(np.int16)

        input_block_idx[k_idx_is_E] = E_in_comming#设置E_neuron

        input_neuron_idx = k_idx.astype(np.uint32)
        for _idx, max_idx in enumerate(_extern_input_k_sizes):
            if _idx != block_idx:
                extern_incomming_idx = (input_block_idx == _idx).nonzero()[0]
                extern_outcomming_idx = get_k_idx(max_idx, extern_incomming_idx.shape[0], -1)
                input_neuron_idx[extern_incomming_idx] = extern_outcomming_idx

        input_channel_offset[k_idx_is_E] = 0 #设置offset
        input_channel_offset[k_idx_is_I] = 2 #设置offset

        input_block_idx -= block_idx
        return input_block_idx, input_neuron_idx, input_channel_offset

    with Thpool() as p:#多线程
        input_block_idx, input_neuron_idx, input_channel_offset = tuple(zip(*p.map(_run, range(s, e))))
    input_block_idx = np.concatenate(input_block_idx)
    input_neuron_idx = np.concatenate(input_neuron_idx)
    input_channel_offset = np.concatenate(input_channel_offset)
    output_neuron_idx = output_neuron_idx.reshape([-1])
    connect_weight = connect_weight.reshape([-1, 2])

    print("done", e - s)
    return output_neuron_idx, input_block_idx, input_neuron_idx, input_channel_offset, connect_weight


In [3]:
import re
import torch
dense=True
bases =None
block_name = re.compile('block_[0-9]*.npz')
path = "C:/Users/lenovo/Desktop/spliking_nn_for_brain_simulation-master/single_small_test/single"
block_length = len([name for name in os.listdir(path) if block_name.fullmatch(name)])#返回指定目录中npz文件的数量
#os.listdir() 方法用于返回指定的文件夹包含的文件或文件夹的名字的列表。这个列表以字母顺序。 它不包括 '.' 和'..' 即使它在文件夹中。
if bases is None:
    bases = [0]
    for i in range(block_length):
        pkl_path = os.path.join(path, "block_{}.npz".format(i))
        file = np.load(pkl_path)
        bases.append(bases[-1] + file["property"].shape[0])

    bases = np.array(bases, dtype=np.int64)

if dense:#如果是一般的矩阵
    weights = []
    indices = []
    sizes = []
    properties = []
    for i in range(block_length):#开始读取每个block中的数据数据
        pkl_path = os.path.join(path, "block_{}.npz".format(i))
        print(pkl_path)
        file = np.load(pkl_path)
        properties.append(file["property"])
        output_neuron_idx, input_block_idx, input_neuron_idx, input_channel_offset, weight = \
            tuple(file[name] for name in ["output_neuron_idx", "input_block_idx", "input_neuron_idx", "input_channel_offset", "weight"])
        idx = np.stack([input_channel_offset.astype(np.uint32),
                        output_neuron_idx.astype(np.uint32),
                        (bases[(input_block_idx + i)] + input_neuron_idx).astype(np.uint32)])
        weight = weight.reshape([-1])
        idx = np.stack([idx, idx], axis=-1)
        idx[0, :, 1] = idx[0, :, 0] + 1
        idx = idx.reshape([3, -1])
        size = [4, np.max(idx[1])+1, np.max(idx[2])+1]

        indices.append(idx)
        weights.append(weight)
        sizes.append(size)
    size = tuple(np.max(np.array(sizes), axis=0)[1:].tolist())
    property = torch.cat([torch.from_numpy(property) for property in properties])
    weight = torch.cat([torch.sparse.FloatTensor(
                torch.from_numpy(indices[i].astype(np.int64)),
                torch.from_numpy(weights[i]),
                torch.Size([4, bases[i+1] - bases[i], size[1]])) for i in range(block_length)], dim=2)
    assert property.shape[0] == weight.shape[1]
    assert weight.shape[2] == weight.shape[1]
    print(property.shape, weight.shape)

C:/Users/lenovo/Desktop/spliking_nn_for_brain_simulation-master/single_small_test/single\block_0.npz
torch.Size([1000, 22]) torch.Size([4, 1000, 1000])


In [4]:
###random innitialize###

In [5]:
def _init(_block_connect_prob,
          _block_node_init_kwards,
          _extern_input_k_sizes,
          _degree,
          _init_min,
          _init_max,
          _dtype,
          _perfix):
    global block_connect_prob
    global block_node_init_kwards
    global extern_input_k_sizes
    global degree
    global init_min
    global init_max
    global dtype
    global perfix

    block_connect_prob = _block_connect_prob
    block_node_init_kwards = _block_node_init_kwards
    extern_input_k_sizes = _extern_input_k_sizes
    degree = _degree
    init_min = _init_min
    init_max = _init_max
    dtype = _dtype
    perfix = _perfix

    np.random.seed()

In [6]:
dtype = 'single'###有关dtype的设置
def connect_for_multi_sparse_block(block_connect_prob, block_node_init_kwards=None,\
                                   E_number=None, I_number=None, degree=int(1e3), \
                                   init_min=0, init_max=1, perfix=None, dtype="single"):
    assert isinstance(block_connect_prob, torch.Tensor) and \
           len(block_connect_prob.shape) == 2 and \
           block_connect_prob.shape[0] == block_connect_prob.shape[1]
    # block_connect_prob should be a [N, N] tensor
    N = block_connect_prob.shape[0]
    block_connect_prob = block_connect_prob.numpy()

    #参数设置整理
    block_node_init_kwards = {} if block_node_init_kwards is None else block_node_init_kwards
    if E_number is not None:
        block_node_init_kwards["E_number"] = E_number
    if I_number is not None:
        block_node_init_kwards["I_number"] = I_number

    if isinstance(block_node_init_kwards, dict):
        extern_input_k_sizes = [block_node_init_kwards["E_number"]] * N
    elif isinstance(block_node_init_kwards, list):
        extern_input_k_sizes = [b["E_number"] for b in block_node_init_kwards]
    else:
        raise ValueError

    print('total {} blocks'.format(N))
    if perfix is None:
        def _out():
            if isinstance(block_node_init_kwards, dict):
                number = [block_node_init_kwards['E_number'] + block_node_init_kwards['I_number']] * N
            elif isinstance(block_node_init_kwards, list):
                number = [b['E_number'] + b['I_number'] for b in block_node_init_kwards]
            else:
                raise ValueError

            bases = np.add.accumulate(np.array(number, dtype=np.int64))
            bases = np.concatenate([np.array([0], dtype=np.int64), bases])

            def prop(i, s, e):
                block_node_init_kward = block_node_init_kwards[i] if isinstance(block_node_init_kwards, list) else block_node_init_kwards#剥壳
                return generate_block_node_property(sub_block_idx=i, s=s, e=e, **block_node_init_kward)

            def conn(i, s, e):
                prob = block_connect_prob[i, :]
                assert np.abs(1 - prob.sum()) < 1e-4
                step = int(1e6)
                for _s in range(s, e, step):
                    _e = min(_s+step, e)
                    output_neuron_idx, input_block_idx, input_neuron_idx, input_neuron_offset, connect_weight =\
                        connect_for_single_sparse_block(i, bases[i+1] - bases[i],
                                                          prob,
                                                          s=_s,
                                                          e=_e,
                                                          extern_input_k_sizes=extern_input_k_sizes,
                                                          degree=degree,
                                                          init_min=init_min,
                                                          init_max=init_max)

                    output_neuron_idx = output_neuron_idx.astype(np.int64)
                    input_neuron_idx = input_neuron_idx.astype(np.int64)

                    output_neuron_idx += bases[i].astype(output_neuron_idx.dtype)
                    input_neuron_idx += bases[i + input_block_idx].astype(input_neuron_idx.dtype)
                    yield output_neuron_idx, input_neuron_idx, input_neuron_offset, connect_weight
            return prop, conn, bases
        return _out
    else:
        if not os.path.exists(perfix):
            os.makedirs(perfix)
        with pool(8, initializer=_init, initargs=(block_connect_prob,
                                               block_node_init_kwards,
                                               extern_input_k_sizes,
                                               degree,
                                               init_min,
                                               init_max,
                                               dtype,
                                               perfix)) as p:
            p.map(_process_dti, range(0, N, 1), chunksize=1)

        print('total done!')
        return

In [8]:
def _process_dti(i):
    global block_connect_prob
    global block_node_init_kwards
    global extern_input_k_sizes
    global degree
    global init_min
    global init_max
    global dtype
    global perfix
    print("dtype: {}".format(dtype))
    def check_if_exist():
        for d in dtype:
            print("d: {}".format(d))
            new_perfix = perfix if len(dtype) == 0 else os.path.join(perfix, d)
            os.makedirs(new_perfix, exist_ok=True)
            if not os.path.exists(os.path.join(new_perfix, "block_{}.npz".format(i))):
                return False
        return True

    if check_if_exist():
        print("skip", i)
        return
    print("processing", i)
    prob = block_connect_prob[i, :]
    assert np.abs(1 - prob.sum()) < 1e-4#概率加起来基本上等于1
    if isinstance(block_node_init_kwards, list):#剥皮
        block_node_init_kward = block_node_init_kwards[i]
    else:
        block_node_init_kward = block_node_init_kwards
    assert isinstance(block_node_init_kward, dict)
    property = generate_block_node_property(sub_block_idx=i, **block_node_init_kward)
    output_neuron_idx, input_block_idx, input_neuron_idx, input_channel_offset, weight \
        = connect_for_single_sparse_block(i, block_node_init_kward['E_number'] + block_node_init_kward['I_number'],
                                             prob,
                                             extern_input_k_sizes=extern_input_k_sizes,
                                             degree=degree,
                                             init_min=init_min,
                                             init_max=init_max)

    if isinstance(dtype, str):
        dtype = [dtype]

    for d in dtype:
        new_perfix = perfix if len(dtype) == 0 else os.path.join(perfix, d)
        os.makedirs(new_perfix, exist_ok=True)

        if d == "single":
            _weight = weight
        elif d == 'half':
            _weight = weight.astype(np.float16)
        else:
            raise ValueError
        np.savez(os.path.join(new_perfix, "block_{}".format(i)),
                 property=np.ascontiguousarray(property),
                 output_neuron_idx=np.ascontiguousarray(output_neuron_idx),
                 input_block_idx=np.ascontiguousarray(input_block_idx),
                 input_neuron_idx=np.ascontiguousarray(input_neuron_idx),
                 input_channel_offset=np.ascontiguousarray(input_channel_offset),
                 weight=np.ascontiguousarray(_weight))

    print("done! ", i)

In [9]:
def connect_for_single_sparse_block(block_idx: object, k: object, extern_input_rate: object, extern_input_k_sizes: object, degree: object = int(1e3),
                                    init_min: object = 0,
                                    init_max: object = 1,
                                    s: object = 0,
                                    e: object = -1) -> object:
    # extern_input_rate only works for E neurons.

    # this function is pertty slow.

    # we assume that the permution

    if e == -1:
        e = k
    assert 0 <= s < e <= k

    print("length:", e - s, "degree:", degree)
    print("generating weight")
    connect_weight = np.random.rand(e - s, degree, 2).astype(np.float32) * (init_max - init_min) + init_min
    print("generating idx")

    E_neuron_thresh = extern_input_k_sizes[block_idx]

    # connect_weight[is_E_neuron == 0] *= 4

    output_neuron_idx = np.broadcast_to(np.arange(s, e, dtype=np.uint32)[:, None], (e-s, degree))

    _extern_input_k_sizes = np.array(extern_input_k_sizes, dtype=np.int64)
    extern_input_rate = np.add.accumulate(extern_input_rate)[:-1]

    @jit(nogil=True, nopython=True)
    def _run(i):
        input_block_idx = np.zeros(degree, dtype=np.int16)#初始化在本脑区内部输入的神经元index集
        input_channel_offset = np.zeros(degree, dtype=np.uint8)#初始化在本脑区内部输入的神经元的channel_offset
        while True:
            k_idx = get_k_idx(k, degree, i)#随机选择degree个非己的神经元作为出度，k为最大的神经元index，degree为出度
            k_idx_is_E = (k_idx < E_neuron_thresh)##与E_neuron产生随机的连接关系？
            k_idx_is_I = (k_idx >= E_neuron_thresh)##与I_neuron产生随机的连接关系？
            if k_idx_is_I.shape[0] > 0:
                break
        # connect_weight[i, k_idx_is_I] *= connect_weight[i, k_idx_is_E].sum() / connect_weight[i, k_idx_is_I].sum()

        input_block_idx[k_idx_is_I] = block_idx#设置I_neuron
        
        #与其他脑区的进入进出
        r = np.random.rand(np.count_nonzero(k_idx_is_E))
        E_in_comming = np.searchsorted(extern_input_rate, r, 'right').astype(np.int16)

        input_block_idx[k_idx_is_E] = E_in_comming#设置E_neuron

        input_neuron_idx = k_idx.astype(np.uint32)
        for _idx, max_idx in enumerate(_extern_input_k_sizes):#与其他脑区的进入进出
            if _idx != block_idx:
                extern_incomming_idx = (input_block_idx == _idx).nonzero()[0]
                extern_outcomming_idx = get_k_idx(max_idx, extern_incomming_idx.shape[0], -1)
                input_neuron_idx[extern_incomming_idx] = extern_outcomming_idx

        input_channel_offset[k_idx_is_E] = 0 #设置offset
        input_channel_offset[k_idx_is_I] = 2 #设置offset

        input_block_idx -= block_idx
        return input_block_idx, input_neuron_idx, input_channel_offset

    with Thpool() as p:#多线程
        input_block_idx, input_neuron_idx, input_channel_offset = tuple(zip(*p.map(_run, range(s, e))))
    input_block_idx = np.concatenate(input_block_idx)
    input_neuron_idx = np.concatenate(input_neuron_idx)
    input_channel_offset = np.concatenate(input_channel_offset)
    output_neuron_idx = output_neuron_idx.reshape([-1])
    connect_weight = connect_weight.reshape([-1, 2])

    print("done", e - s)
    return output_neuron_idx, input_block_idx, input_neuron_idx, input_channel_offset, connect_weight

In [10]:
def generate_block_node_property(E_number=int(8e5),
                            I_number=int(2e5),
                            I_extern_Input = 0,
                            sub_block_idx=0,
                            C = 1,
                            T_ref = 5,
                            g_Li = 0.001,
                            V_L = -75,
                            V_th = -50,
                            V_reset = -65,
                            g_ui = (5/275, 5/4000, 3/30, 3/730),
                            V_ui = (0, 0, -70, -100),
                            tao_ui = (2, 40, 10, 50),
                            s = 0, e = -1):

    # each node contain such property:
    #          E/I, blocked_in_stat, I_extern_Input, sub_block_idx, C, T_ref, g_Li, V_L, V_th, V_reset, g_ui, V_ui, tao_ui
    #   size:  1,   1,               1,                1,           1, 1,     1,    1,   1,    1,       4     4,    4
    #   dtype: b,   b,               f,                i,           f, f,     f,    f,   f,    f,       f,    f,    f
    # b means bool(although storage as float), f means float.

    # this function support broadcast, e.g, C can be a scalar for a total block or a [E_number, I_number] tensor for total nodes.
    if e == -1:
        e = E_number + I_number
    assert 0 <= s < e <= E_number + I_number

    property = np.zeros([e - s, 22], dtype=np.float32)
    E_thresh = E_number - s if E_number > s else 0
    property[:E_thresh, 0] = 1
    property[E_thresh:, 0] = 0

    property[:, 1] = 0

    property[:, 2] = I_extern_Input

    property[:, 3] = sub_block_idx
    property[:, 4] = C
    property[:, 5] = T_ref
    property[:, 6] = g_Li
    property[:, 7] = V_L
    property[:, 8] = V_th
    property[:, 9] = V_reset

    g_ui = g_ui if isinstance(g_ui, np.ndarray) else np.array(g_ui)
    property[:, 10:14] = g_ui

    V_ui = V_ui if isinstance(V_ui, np.ndarray) else np.array(V_ui)
    property[:, 14:18] = V_ui

    tao_ui = tao_ui if isinstance(tao_ui, np.ndarray) else np.array(tao_ui)

    property[:, 18:22] = tao_ui

    return property

In [11]:
@jit(nogil=True, nopython=True)
def get_k_idx(max_k, num, except_idx):
    if except_idx < 0:
        assert num <= max_k
        if num == max_k:
            return np.arange(0, max_k)
    elif except_idx is not None:
        assert num < max_k
        if num == max_k - 1:
            return np.concatenate((np.arange(0, except_idx), np.arange(except_idx+1, num)))#输出除了except_idx之外的其他连续数列，从0到num

    while True:
        k_idx = np.unique(np.random.randint(0, max_k, num*2))#随机生成一个2*num规模的随机整数列，去掉重复值，取值为0-max_k
        k_idx = k_idx[np.random.permutation(k_idx.shape[0])]#对上述产生的序列进行随机打乱
        if except_idx is not None:
            k_idx = k_idx[k_idx != except_idx] #删掉except_idx
        k_idx = k_idx[:num]#截取num个元素
        if k_idx.shape[0] == num:
            break
    return k_idx

In [10]:
(block_connect_prob, block_node_init_kwards=None,E_number=None, I_number=None, degree=int(1e3),init_min=0, init_max=1, perfix=None, dtype="single")
connect_for_multi_sparse_block(block_connect_prob=prob, 
                               block_node_init_kwards={'g_Li': 0.003,
                                      'g_ui': (0.022178268060088158,
                                               0.0013867146335542202,
                                               0.09227322041988373,
                                               0.004128940403461456),
                                      "V_reset": -65,
                                      "V_th": -50},
                               E_number=int(8e2), 
                               I_number=int(2e2), 
                               degree=int(20), 
                               init_min=0, 
                               init_max=1, 
                               perfix="./single_small/")

SyntaxError: invalid syntax (<ipython-input-10-c24d241a96bd>, line 1)

In [12]:
block_connect_prob=torch.tensor([[1.]])
block_node_init_kwards={'g_Li': 0.003,
'g_ui': (0.022178268060088158,
0.0013867146335542202,
0.09227322041988373,
0.004128940403461456),
"V_reset": -65,
"V_th": -50};
E_number=int(8e2)
I_number=int(2e2)
degree=int(20)
init_min=0
init_max=1
perfix="./my_single_small/"

In [13]:
assert isinstance(block_connect_prob, torch.Tensor) and \
       len(block_connect_prob.shape) == 2 and \
       block_connect_prob.shape[0] == block_connect_prob.shape[1]

In [14]:
# block_connect_prob should be a [N, N] tensor
N = block_connect_prob.shape[0]
block_connect_prob = block_connect_prob.numpy()

In [15]:
#参数设置整理
block_node_init_kwards = {} if block_node_init_kwards is None else block_node_init_kwards
if E_number is not None:
    block_node_init_kwards["E_number"] = E_number
if I_number is not None:
    block_node_init_kwards["I_number"] = I_number

if isinstance(block_node_init_kwards, dict):
    extern_input_k_sizes = [block_node_init_kwards["E_number"]] * N
elif isinstance(block_node_init_kwards, list):
    extern_input_k_sizes = [b["E_number"] for b in block_node_init_kwards]
else:
    raise ValueError

In [16]:
if not os.path.exists(perfix):
    os.makedirs(perfix)

In [148]:
dtype = ['single']

In [ ]:
dtype = ['single']
if not os.path.exists(perfix):
    os.makedirs(perfix)
with pool(1, initializer=_init, initargs=(block_connect_prob,
                                       block_node_init_kwards,
                                       extern_input_k_sizes,
                                       degree,
                                       init_min,
                                       init_max,
                                       dtype,
                                       perfix)) as p:
    p.map(_process_dti, range(0, N, 1), chunksize=1)

print('total done!')

In [18]:
_init(block_connect_prob,
block_node_init_kwards,
extern_input_k_sizes,
degree,
init_min,
init_max,
dtype,
perfix)

In [28]:
_process_dti(0)

dtype: ['single']
d: single
processing 0
length: 1000 degree: 20
generating weight
generating idx
done 1000
done!  0


In [20]:
def check_if_exist():
    for d in dtype:
        new_perfix = perfix if len(dtype) == 0 else os.path.join(perfix, d)
        os.makedirs(new_perfix, exist_ok=True)
        if not os.path.exists(os.path.join(new_perfix, "block_{}.npz".format(i))):
            return False
    return True

In [21]:
for d in dtype:
    new_perfix = perfix if len(dtype) == 0 else os.path.join(perfix, d)
    os.makedirs(new_perfix, exist_ok=True)
    tof =  os.path.exists(os.path.join(new_perfix, "block_{}.npz".format(i)))

In [22]:

print("processing", i)
prob = block_connect_prob[i, :]

processing 0


In [23]:
if isinstance(block_node_init_kwards, list):#剥皮
    block_node_init_kward = block_node_init_kwards[i]
else:
    block_node_init_kward = block_node_init_kwards

In [24]:
assert isinstance(block_node_init_kward, dict)

In [25]:
property = generate_block_node_property(sub_block_idx=i, **block_node_init_kward)
output_neuron_idx, input_block_idx, input_neuron_idx, input_channel_offset, weight \
    = connect_for_single_sparse_block(i, block_node_init_kward['E_number'] + block_node_init_kward['I_number'],
                                         prob,
                                         extern_input_k_sizes=extern_input_k_sizes,
                                         degree=degree,
                                         init_min=init_min,
                                         init_max=init_max)

length: 1000 degree: 20
generating weight
generating idx
done 1000


In [147]:
#做了修改，可以跑
("processing", i)
prob = block_connect_prob[i, :]
assert np.abs(1 - prob.sum()) < 1e-4#概率加起来基本上等于1
if isinstance(block_node_init_kwards, list):#剥皮
    block_node_init_kward = block_node_init_kwards[i]
else:
    block_node_init_kward = block_node_init_kwards
assert isinstance(block_node_init_kward, dict)
property = generate_block_node_property(sub_block_idx=i, **block_node_init_kward)
output_neuron_idx, input_block_idx, input_neuron_idx, input_channel_offset, weight \
    = connect_for_single_sparse_block(i, block_node_init_kward['E_number'] + block_node_init_kward['I_number'],
                                         prob,
                                         extern_input_k_sizes=extern_input_k_sizes,
                                         degree=degree,
                                         init_min=init_min,
                                         init_max=init_max)

if isinstance(dtype, str):#要小心dtype的形式
    dtype = [dtype]

for d in dtype:#目录创建
    new_perfix = perfix if len(dtype) == 0 else os.path.join(perfix, d)
    os.makedirs(new_perfix, exist_ok=True)

    if d == "single":
        _weight = weight#没有half意味着dtype=float32
    elif d == 'half':
        _weight = weight.astype(np.float16)#half意味着数据类型为float16
    else:
        raise ValueError
    np.savez(os.path.join(new_perfix, "block_{}".format(i)),#储存各参数
             property=np.ascontiguousarray(property),
             output_neuron_idx=np.ascontiguousarray(output_neuron_idx),
             input_block_idx=np.ascontiguousarray(input_block_idx),
             input_neuron_idx=np.ascontiguousarray(input_neuron_idx),
             input_channel_offset=np.ascontiguousarray(input_channel_offset),
             weight=np.ascontiguousarray(_weight))

print("done! ", i)

processing 0
length: 1000 degree: 20
generating weight
generating idx
done 1000
done!  0


In [146]:
#做了修改
def connect_for_single_sparse_block(block_idx, k, extern_input_rate, extern_input_k_sizes, degree = int(1e3),
                                    init_min = 0,
                                    init_max = 1,
                                    s = 0,
                                    e = -1):
    # extern_input_rate only works for E neurons.

    # this function is pertty slow.

    # we assume that the permution

    if e == -1:
        e = k
    assert 0 <= s < e <= k

    print("length:", e - s, "degree:", degree)
    print("generating weight")
    connect_weight = np.random.rand(e - s, degree, 2).astype(np.float32) * (init_max - init_min) + init_min
    print("generating idx")

    E_neuron_thresh = extern_input_k_sizes[block_idx]

    # connect_weight[is_E_neuron == 0] *= 4

    output_neuron_idx = np.broadcast_to(np.arange(s, e, dtype=np.uint32)[:, None], (e-s, degree))

    _extern_input_k_sizes = np.array(extern_input_k_sizes, dtype=np.int64)
    extern_input_rate = np.add.accumulate(extern_input_rate)[:-1]

    @jit(nogil=True, nopython=True)
    def _run(i):
        input_block_idx = np.zeros(degree, dtype=np.int16)#初始化
        input_channel_offset = np.zeros(degree, dtype=np.uint8)#初始化
        while True:
            k_idx = get_k_idx(k, degree, i)#不能与自己连接，所以排除i
            k_idx_is_E = (k_idx < E_neuron_thresh)##与E_neuron产生随机的连接关系？
            k_idx_is_I = (k_idx >= E_neuron_thresh)##与I_neuron产生随机的连接关系？
            if k_idx_is_I.shape[0] > 0:
                break
        # connect_weight[i, k_idx_is_I] *= connect_weight[i, k_idx_is_E].sum() / connect_weight[i, k_idx_is_I].sum()

        input_block_idx[k_idx_is_I] = block_idx#设置I_neuron

        r = np.random.rand(np.count_nonzero(k_idx_is_E))
        E_in_comming = np.searchsorted(extern_input_rate, r, 'right').astype(np.int16)

        input_block_idx[k_idx_is_E] = E_in_comming#设置E_neuron

        input_neuron_idx = k_idx.astype(np.uint32)
        for _idx, max_idx in enumerate(_extern_input_k_sizes):
            if _idx != block_idx:
                extern_incomming_idx = (input_block_idx == _idx).nonzero()[0]
                extern_outcomming_idx = get_k_idx(max_idx, extern_incomming_idx.shape[0], -1)
                input_neuron_idx[extern_incomming_idx] = extern_outcomming_idx

        input_channel_offset[k_idx_is_E] = 0 #设置offset
        input_channel_offset[k_idx_is_I] = 2 #设置offset

        input_block_idx -= block_idx
        return input_block_idx, input_neuron_idx, input_channel_offset

    input_block_idx, input_neuron_idx, input_channel_offset = list(zip(list(map(_run, range(s, e)))[0]))#核心计算过程
    output_neuron_idx = output_neuron_idx.reshape([-1])
    connect_weight = connect_weight.reshape([-1, 2])

    print("done", e - s)
    return output_neuron_idx, input_block_idx, input_neuron_idx, input_channel_offset, connect_weight

In [136]:
result00=list(map(_run, range(0, 2)))

In [142]:
list(zip(result00[0]))

[(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=int16),),
 (array([851, 499, 682, 395, 602, 349, 858, 178, 128, 591, 297, 640, 883,
         990, 667,  54, 470, 942, 892,  45], dtype=uint32),),
 (array([2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0],
        dtype=uint8),)]

In [61]:
range(s, e)

range(0, 1000)

In [ ]:
block_idx, k, extern_input_rate, extern_input_k_sizes, degree = int(1e3),
                                    init_min = 0,
                                    init_max = 1,
                                    s = 0,
                                    e = -1

connect_for_single_sparse_block(i, block_node_init_kward['E_number'] + block_node_init_kward['I_number'],
                                         prob,
                                         extern_input_k_sizes=extern_input_k_sizes,
                                         degree=degree,
                                         init_min=init_min,
                                         init_max=init_max)

In [34]:
block_idx =i
k=block_node_init_kward['E_number'] + block_node_init_kward['I_number']  
extern_input_rate = prob
extern_input_k_sizes=extern_input_k_sizes
degree=degree
init_min=init_min
init_max=init_max
s = 0
e = -1

In [35]:
if e == -1:
    e = k
assert 0 <= s < e <= k

print("length:", e - s, "degree:", degree)
print("generating weight")
connect_weight = np.random.rand(e - s, degree, 2).astype(np.float32) * (init_max - init_min) + init_min
print("generating idx")

E_neuron_thresh = extern_input_k_sizes[block_idx]

length: 1000 degree: 20
generating weight
generating idx


In [44]:
# connect_weight[is_E_neuron == 0] *= 4

output_neuron_idx = np.broadcast_to(np.arange(s, e, dtype=np.uint32)[:, None], (e-s, degree))

_extern_input_k_sizes = np.array(extern_input_k_sizes, dtype=np.int64)
extern_input_rate = np.add.accumulate(extern_input_rate)[:-1]

In [62]:
@jit(nogil=True, nopython=True)
def _run(i):
    input_block_idx = np.zeros(degree, dtype=np.int16)#初始化
    input_channel_offset = np.zeros(degree, dtype=np.uint8)#初始化
    while True:
        k_idx = get_k_idx(k, degree, i)#不能与自己连接，所以排除i
        k_idx_is_E = (k_idx < E_neuron_thresh)##与E_neuron产生随机的连接关系？
        k_idx_is_I = (k_idx >= E_neuron_thresh)##与I_neuron产生随机的连接关系？
        if k_idx_is_I.shape[0] > 0:
            break
    # connect_weight[i, k_idx_is_I] *= connect_weight[i, k_idx_is_E].sum() / connect_weight[i, k_idx_is_I].sum()

    input_block_idx[k_idx_is_I] = block_idx#设置I_neuron

    r = np.random.rand(np.count_nonzero(k_idx_is_E))
    E_in_comming = np.searchsorted(extern_input_rate, r, 'right').astype(np.int16)

    input_block_idx[k_idx_is_E] = E_in_comming#设置E_neuron

    input_neuron_idx = k_idx.astype(np.uint32)
    for _idx, max_idx in enumerate(_extern_input_k_sizes):
        if _idx != block_idx:
            extern_incomming_idx = (input_block_idx == _idx).nonzero()[0]
            extern_outcomming_idx = get_k_idx(max_idx, extern_incomming_idx.shape[0], -1)
            input_neuron_idx[extern_incomming_idx] = extern_outcomming_idx

    input_channel_offset[k_idx_is_E] = 0 #设置offset
    input_channel_offset[k_idx_is_I] = 2 #设置offset

    input_block_idx -= block_idx
    return input_block_idx, input_neuron_idx, input_channel_offset


In [60]:
np.add.accumulate(extern_input_rate)

array([], dtype=float32)

In [52]:
np.arange(0, 2, dtype=np.uint32)[:, None]

array([[0],
       [1]], dtype=uint32)

In [63]:
input_block_idx = np.zeros(degree, dtype=np.int16)#初始化
input_channel_offset = np.zeros(degree, dtype=np.uint8)#初始化

In [65]:
while True:
    k_idx = get_k_idx(k, degree, i)#不能与自己连接，所以排除i
    k_idx_is_E = (k_idx < E_neuron_thresh)##与E_neuron产生随机的连接关系？
    k_idx_is_I = (k_idx >= E_neuron_thresh)##与I_neuron产生随机的连接关系？
    if k_idx_is_I.shape[0] > 0:
        break

In [81]:
input_block_idx[k_idx_is_I] = block_idx#设置I_neuron

r = np.random.rand(np.count_nonzero(k_idx_is_E))
E_in_comming = np.searchsorted(extern_input_rate, r, 'right').astype(np.int16)

input_block_idx[k_idx_is_E] = E_in_comming#设置E_neuron

input_neuron_idx = k_idx.astype(np.uint32)

In [91]:
k_idx

array([921, 199, 628, 704, 952, 734, 590, 705, 493, 411, 329, 944, 922,
        90,  21, 538, 458, 282, 460, 569], dtype=int64)

In [93]:
for _idx, max_idx in enumerate(_extern_input_k_sizes):
    if _idx != block_idx:
        extern_incomming_idx = (input_block_idx == _idx).nonzero()[0]
        extern_outcomming_idx = get_k_idx(max_idx, extern_incomming_idx.shape[0], -1)
        input_neuron_idx[extern_incomming_idx] = extern_outcomming_idx

In [95]:
input_channel_offset[k_idx_is_E] = 0 #设置offset
input_channel_offset[k_idx_is_I] = 2 #设置offset

input_block_idx -= block_idx

0

In [ ]:
@jit(nogil=True, nopython=True)
def get_k_idx(max_k, num, except_idx):
    if except_idx < 0:
        assert num <= max_k
        if num == max_k:
            return np.arange(0, max_k)
    elif except_idx is not None:
        assert num < max_k
        if num == max_k - 1:
            return np.concatenate((np.arange(0, except_idx), np.arange(except_idx+1, num)))#输出除了except_idx之外的其他连续数列，从0到num

    while True:
        k_idx = np.unique(np.random.randint(0, max_k, num*2))#随机生成一个2*num规模的随机整数列，去掉重复值，取值为0-max_k
        k_idx = k_idx[np.random.permutation(k_idx.shape[0])]#对上述产生的序列进行随机打乱
        if except_idx is not None:
            k_idx = k_idx[k_idx != except_idx] #删掉except_idx
        k_idx = k_idx[:num]#截取num个元素
        if k_idx.shape[0] == num:
            break
    return k_idx

In [74]:
k_idx = get_k_idx(k, degree, i)#不能与自己连接，所以排除i
k_idx_is_I.shape[0]

20

In [ ]:
input_block_idx = np.zeros(degree, dtype=np.int16)#初始化
input_channel_offset = np.zeros(degree, dtype=np.uint8)#初始化
while True:
    k_idx = get_k_idx(k, degree, i)#不能与自己连接，所以排除i
    k_idx_is_E = (k_idx < E_neuron_thresh)##与E_neuron产生随机的连接关系？
    k_idx_is_I = (k_idx >= E_neuron_thresh)##与I_neuron产生随机的连接关系？
    if k_idx_is_I.shape[0] > 0:
        break
# connect_weight[i, k_idx_is_I] *= connect_weight[i, k_idx_is_E].sum() / connect_weight[i, k_idx_is_I].sum()

input_block_idx[k_idx_is_I] = block_idx#设置I_neuron

r = np.random.rand(np.count_nonzero(k_idx_is_E))
E_in_comming = np.searchsorted(extern_input_rate, r, 'right').astype(np.int16)

input_block_idx[k_idx_is_E] = E_in_comming#设置E_neuron

input_neuron_idx = k_idx.astype(np.uint32)
input_channel_offset[k_idx_is_E] = 0 #设置offset
input_channel_offset[k_idx_is_I] = 2 #设置offset